# Data
Here is all of the data that I will be using for my capstone project analyzing income inequality disparities within Boston and how this manifests in its neighborhoods. I will be using a combination of Four square location data as well as Boston income data

### Per Capita Income by Boston Neighborhood

In [1]:
import numpy as np
import folium
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json

from geopy.geocoders import Nominatim

import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors

from sklearn.cluster import KMeans

from bs4 import BeautifulSoup

import xml

import folium

print('Libraries Imported.')

Libraries Imported.


In [9]:
url_mass = "https://en.wikipedia.org/wiki/Boston"
wiki_mass_url = requests.get(url_mass)
wiki_mass_data = pd.read_html(wiki_mass_url.text)
wiki_mass_data = wiki_mass_data[9]
wiki_mass_data

,Rank,ZIP code (ZCTA),Per capitaincome,Medianhouseholdincome,Medianfamilyincome,Population,Number ofhouseholds
0,1.0,02110 (Financial District),"$152,007","$123,795","$196,518",1486,981
1,2.0,02199 (Prudential Center),"$151,060","$107,159","$146,786",1290,823
2,3.0,02210 (Fort Point),"$93,078","$111,061","$223,411",1905,1088
3,4.0,02109 (North End),"$88,921","$128,022","$162,045",4277,2190
4,5.0,02116 (Back Bay/Bay Village),"$81,458","$87,630","$134,875",21318,10938
5,6.0,02108 (Beacon Hill/Financial District),"$78,569","$95,753","$153,618",4155,2337
6,7.0,02114 (Beacon Hill/West End),"$65,865","$79,734","$169,107",11933,6752
7,8.0,02111 (Chinatown/Financial District/Leather Di...,"$56,716","$44,758","$88,333",7616,3390
8,9.0,02129 (Charlestown),"$56,267","$89,105","$98,445",17052,8083
9,10.0,02467 (Chestnut Hill),"$53,382","$113,952","$148,396",22796,6351


### Cleaning Up the Table to Make the Data Clearer

Starting by dropping the Percapitaincome and Medianfamilyincome columns. Then sorted by median household income.

In [10]:
wiki_mass_data.drop(['Per capitaincome', 'Medianfamilyincome'], axis=1,inplace=True)
wiki_mass_data.head()

,Rank,ZIP code (ZCTA),Medianhouseholdincome,Population,Number ofhouseholds
0,1.0,02110 (Financial District),"$123,795",1486,981
1,2.0,02199 (Prudential Center),"$107,159",1290,823
2,3.0,02210 (Fort Point),"$111,061",1905,1088
3,4.0,02109 (North End),"$128,022",4277,2190
4,5.0,02116 (Back Bay/Bay Village),"$87,630",21318,10938


In [15]:
wiki_mass_data.sort_values(by='Medianhouseholdincome', ascending=True)

,Rank,ZIP code (ZCTA),Medianhouseholdincome,Population,Number ofhouseholds
1,2.0,02199 (Prudential Center),"$107,159",1290,823
2,3.0,02210 (Fort Point),"$111,061",1905,1088
9,10.0,02467 (Chestnut Hill),"$113,952",22796,6351
0,1.0,02110 (Financial District),"$123,795",1486,981
3,4.0,02109 (North End),"$128,022",4277,2190
28,25.0,"02115 (Back Bay, Longwood, Museum of Fine Arts...","$23,677",29178,9958
31,28.0,02119 (Roxbury),"$27,051",24237,9769
32,29.0,02121 (Dorchester-Mount Bowdoin),"$30,419",26801,9739
30,27.0,02215 (Fenway-Kenmore),"$30,823",23719,7995
33,30.0,02120 (Mission Hill),"$32,367",13217,4509


In [19]:
wiki_mass_data = wiki_mass_data.rename(index=str, columns={'Medianhouseholdincome': 'Median Household Income', 'ZIP code (ZCTA)': 'Zip Code','Number ofhouseholds':'Number of Households'})
wiki_mass_data.head()

,Rank,Zip Code,Median Household Income,Population,Number of Households
0,1.0,02110 (Financial District),"$123,795",1486,981
1,2.0,02199 (Prudential Center),"$107,159",1290,823
2,3.0,02210 (Fort Point),"$111,061",1905,1088
3,4.0,02109 (North End),"$128,022",4277,2190
4,5.0,02116 (Back Bay/Bay Village),"$87,630",21318,10938


In [16]:
address = 'Boston, MA'

geolocator = Nominatim(user_agent='Bos_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The Geographical Coordinates of Boston are {}, {}.'.format(latitude, longitude))

The Geographical Coordinates of Boston are 42.3602534, -71.0582912.


So this concludes the data that I will be using for this project. Using these coordinates and zip code data, I will use the foursquare location data to analyze each neighborhood to see the top venues, and then shall compare and contrast between the res